In [1]:
import os
import gc
from tqdm.auto import tqdm
import transformers
import numpy as np 
import pandas as pd 
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import plotly.express as px #graphing
import plotly.graph_objects as go #graphing
from plotly.subplots import make_subplots #graphing
import plotly.figure_factory as ff #graphing

from transformers import get_polynomial_decay_schedule_with_warmup,get_cosine_schedule_with_warmup,get_linear_schedule_with_warmup
from transformers.tokenization_utils_base import BatchEncoding, PreTrainedTokenizerBase
from transformers import DataCollatorWithPadding,DataCollatorForTokenClassification


from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs
from datasets import concatenate_datasets,load_dataset,load_from_disk

from sklearn.metrics import log_loss

from transformers import AutoModel, AutoTokenizer, AdamW, DataCollatorWithPadding

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import time
import warnings
import collections
from termcolor import colored

from torch.optim import lr_scheduler
warnings.filterwarnings("ignore")



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]#.detach().to('cpu').numpy()
        y_pred = y_preds[:,i]#.detach().to('cpu').numpy()
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def score_loss(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return {
        'mcrmse_score' : mcrmse_score,
        'Content_score' : scores[0],
        'Wording_score' : scores[1]
    }

In [3]:
oof_df = pd.read_csv('/kaggle/input/commonlit-deberta-base/oof_df.csv')
#oof_df = pd.read_csv('/kaggle/input/commonlit-deberta-hidden-layers-mean/oof_df.csv')
print(oof_df.shape)
oof_df.head()

(7165, 4)


,content,wording,pred_content,pred_wording
0,1.658802,-0.093150,0.836136,0.151125
1,-1.638511,-0.911973,-1.300229,-0.716050
2,0.205682,0.380538,0.192420,0.089422
3,-1.355562,-0.955801,-1.039375,-0.897786
4,0.205682,0.380538,-0.006736,0.227909


In [4]:
s = score_loss(np.array(oof_df[['content' , 'wording']]) , np.array(oof_df[['pred_content' , 'pred_wording']]))
s

{'mcrmse_score': 0.46615662019200665,
 'Content_score': 0.39647407605816687,
 'Wording_score': 0.5358391643258464}

In [5]:
import torch
import torch.nn as nn
import transformers
from transformers import (
    AutoModel, AutoConfig, 
    AutoTokenizer, logging
)

In [6]:
# ====================================================
# Utils
# ====================================================


class cfg:
    select = 'large'
    model_name = f'/kaggle/input/deberta-v3-{select}/deberta-v3-{select}'
    only_model_name = f'deberta-v3-{select}'
    fold = 4
    batch_size = 32
    freezing = True
    max_len = 1024
    pooling = 'GemText'
    path = '/kaggle/input/commonlit-deberta-base/'
    #path = '/kaggle/input/commonlit-deberta-hidden-layers-mean/'
    #path = '/kaggle/input/commonlit-baseline/'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    seed = 42
cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.path+'tokenizer/')
cfg.device , cfg.model_name

(device(type='cuda'), '/kaggle/input/deberta-v3-large/deberta-v3-large')

In [7]:

def get_logger(filename='Inference'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(cfg.seed)


LOGGER.info(f"max_len: {cfg.max_len}")
LOGGER.info(f"batch_size: {cfg.batch_size}")
LOGGER.info(f"Model name: {cfg.only_model_name}")

max_len: 1024
batch_size: 32
Model name: deberta-v3-large


In [8]:
prompts_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv')
summary_test = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv')
submission = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv')

print(f"Prompts Test shape: {prompts_test.shape}")
display(prompts_test.head())
print(f"Summary Test shape: {summary_test.shape}")
display(summary_test.head())
print(f"Submission shape: {submission.shape}")
display(submission.head())

Prompts Test shape: (2, 4)


,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


Summary Test shape: (4, 3)


,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


Submission shape: (4, 3)


,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


In [9]:
test = prompts_test.merge(summary_test, on="prompt_id")
test

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text
0,abc123,Summarize...,Example Title 1,Heading\nText...,000000ffffff,Example text 1
1,abc123,Summarize...,Example Title 1,Heading\nText...,222222cccccc,Example text 3
2,def789,Summarize...,Example Title 2,Heading\nText...,111111eeeeee,Example text 2
3,def789,Summarize...,Example Title 2,Heading\nText...,333333dddddd,Example text 4


In [10]:
test['full_text']=test['prompt_question'] +" " + cfg.tokenizer.sep_token +" "+test['text']
test['full_text']

0    Summarize... [SEP] Example text 1
1    Summarize... [SEP] Example text 3
2    Summarize... [SEP] Example text 2
3    Summarize... [SEP] Example text 4
Name: full_text, dtype: object

In [11]:
def odd_layer_freeze(module):
    for i in range(1,24,2):
        for n,p in module.encoder.layer[i].named_parameters():
            p.requires_grad = False
            
def even_layer_freeze(module):
    for i in range(0,24,2):
        for n,p in module.encoder.layer[i].named_parameters():
            p.requires_grad = False
            
def top_half_layer_freeze(module):
    for i in range(0,13,1):
        for n,p in module.encoder.layer[i].named_parameters():
            p.requires_grad = False

def bottom_half_layer_freeze(module):
    for i in range(13,14,1):
        for n,p in module.encoder.layer[i].named_parameters():
            p.requires_grad = False

In [12]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    
    
class GeMText(nn.Module):
    def __init__(self, dim = 1, p=3, eps=1e-6):
        super(GeMText, self).__init__()
        self.dim = dim
        self.p = Parameter(torch.ones(1) * p)
        self.eps = eps
        self.feat_mult = 1

    def forward(self, last_hidden_state, attention_mask):
        attention_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.shape)
        x = (last_hidden_state.clamp(min=self.eps) * attention_mask_expanded).pow(self.p).sum(self.dim)
        ret = x / attention_mask_expanded.sum(self.dim).clip(min=self.eps)
        ret = ret.pow(1 / self.p)
        return ret



def get_pooling_layer():
    if cfg.pooling == 'Mean':
        return MeanPooling()
    
    elif cfg.pooling == 'Max':
        return MaxPooling()
    
    elif cfg.pooling == 'MeanMax':
        return MeanMaxPooling()
    
    elif cfg.pooling == 'GemText':
        return GeMText()


print(get_pooling_layer())

GeMText()


In [13]:


class BaselineModel(nn.Module):
    def __init__(self, model_name ):
        super(BaselineModel, self).__init__()
        
        self.model = AutoModel.from_pretrained(cfg.model_name)
        self.config = AutoConfig.from_pretrained(cfg.model_name)
        #self.drop = nn.Dropout(p=0.2)
        self.pooler = get_pooling_layer()

        if cfg.pooling == 'MeanMax':
            self.fc = nn.Linear(2*self.config.hidden_size, 2)
        else:
            self.fc = nn.Linear(self.config.hidden_size, 2)
            
        
        self._init_weights(self.fc)
        
        if cfg.freezing:
            top_half_layer_freeze(self.model)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
           
    def forward(self, ids, mask):
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        #out = self.drop(out)
        outputs = self.fc(out)
        return outputs

In [14]:
'''
class deberta_hs_Mean(nn.Module):
    def __init__(self, model_name ):
        super(deberta_hs_Mean, self).__init__()
        
        self.model_config = AutoConfig.from_pretrained(cfg.model_name)
        self.model_config.update(
            {
                "output_hidden_states": True,
                "hidden_dropout_prob": 0,
                "add_pooling_layer": False,
                "num_labels": 2,
                 "attention_probs_dropout_prob":0.0 
            }
        )
        self.model = AutoModel.from_pretrained(cfg.model_name, config=self.model_config)
        
        self.fc = nn.Linear(self.model_config.hidden_size, 2)
        
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.model_config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.model_config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def forward(self, ids, mask):
        outputs = self.model(input_ids=ids,attention_mask=mask)
        
        h1 = outputs[1][-1][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h2 = outputs[1][-2][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h3 = outputs[1][-3][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h4 = outputs[1][-4][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h5 = outputs[1][-5][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h6 = outputs[1][-6][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h7 = outputs[1][-7][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h8 = outputs[1][-8][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h9 = outputs[1][-9][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h10 = outputs[1][-10][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h11 = outputs[1][-11][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        h12 = outputs[1][-12][:,0,:].reshape(-1,1,self.model_config.hidden_size)
        
        all_cat_mean = torch.mean(
        torch.cat([ h1, h2, h3, h4, h5, h6,h7,h8,h9,h10,h11,h12], 1)
        ,1).reshape(-1,1,self.model_config.hidden_size)
        
        seq_out = torch.mean(torch.cat([ outputs.last_hidden_state[:,0,:].reshape(-1,1,self.model_config.hidden_size),
        all_cat_mean] , 1),1)
        
        out = self.fc(seq_out)
        return out
'''

'\nclass deberta_hs_Mean(nn.Module):\n    def __init__(self, model_name ):\n        super(deberta_hs_Mean, self).__init__()\n        \n        self.model_config = AutoConfig.from_pretrained(cfg.model_name)\n        self.model_config.update(\n            {\n                "output_hidden_states": True,\n                "hidden_dropout_prob": 0,\n                "add_pooling_layer": False,\n                "num_labels": 2,\n                 "attention_probs_dropout_prob":0.0 \n            }\n        )\n        self.model = AutoModel.from_pretrained(cfg.model_name, config=self.model_config)\n        \n        self.fc = nn.Linear(self.model_config.hidden_size, 2)\n        \n        self._init_weights(self.fc)\n        \n    def _init_weights(self, module):\n        if isinstance(module, nn.Linear):\n            module.weight.data.normal_(mean=0.0, std=self.model_config.initializer_range)\n            if module.bias is not None:\n                module.bias.data.zero_()\n        elif isinst

In [15]:
'''
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    
class BaselineModel(nn.Module):
    def __init__(self, model_name):
        super(BaselineModel, self).__init__()
        
        self.model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        #self.drop = nn.Dropout(p=0.2)
        self.pooler = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 2)
        
        self._init_weights(self.fc)        
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
           
    def forward(self, ids, mask):
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.pooler(out.last_hidden_state, mask)
        #out = self.drop(out)
        outputs = self.fc(out)
        return outputs

'''

'\nclass MeanPooling(nn.Module):\n    def __init__(self):\n        super(MeanPooling, self).__init__()\n        \n    def forward(self, last_hidden_state, attention_mask):\n        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()\n        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)\n        sum_mask = input_mask_expanded.sum(1)\n        sum_mask = torch.clamp(sum_mask, min=1e-9)\n        mean_embeddings = sum_embeddings / sum_mask\n        return mean_embeddings\n    \nclass BaselineModel(nn.Module):\n    def __init__(self, model_name):\n        super(BaselineModel, self).__init__()\n        \n        self.model = AutoModel.from_pretrained(model_name)\n        self.config = AutoConfig.from_pretrained(model_name)\n        #self.drop = nn.Dropout(p=0.2)\n        self.pooler = MeanPooling()\n        self.fc = nn.Linear(self.config.hidden_size, 2)\n        \n        self._init_weights(self.fc)        \n        \n    def 

In [16]:
class TestDataset(Dataset):
    def __init__(self,df):
        self.df = df
        self.tokenizer = cfg.tokenizer
        self.max_len = cfg.max_len
        self.pq = df['prompt_question'].values
        self.text = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self , index):
        pq   =   self.pq[index]
        text =   self.text[index]
        full_text = pq+" " + self.tokenizer.sep_token +" "+text
        inputs = self.tokenizer.encode_plus(
                        full_text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                        
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long)
        }
    
def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:, :mask_len]
    return inputs

In [17]:
@torch.no_grad()
def test_run(model , loader):
    
    
    model.eval()
       
    preds = []
    bar = tqdm(enumerate(loader), total=len(loader))
    for idx , data in bar:
        inputs = collate(data)
        ids   =  inputs['input_ids'].to(cfg.device, dtype = torch.long)
        mask  =  inputs['attention_mask'].to(cfg.device, dtype = torch.long)
        y_preds = model(ids , mask)
        preds.append(y_preds.to('cpu').numpy())
    
    predictions = np.concatenate(preds)
    
    return predictions
    

In [18]:
test_dataset = TestDataset(test)
test_loader = DataLoader(test_dataset , batch_size=cfg.batch_size ,num_workers=2, shuffle=False, pin_memory=True)

In [19]:

final_preds = []
for fold in range(cfg.fold):
    print('******** fold' , fold , '********')
    
    model  = BaselineModel(cfg.model_name).to(cfg.device)
    model.load_state_dict(torch.load(f"/kaggle/input/commonlit-deberta-base/deberta-v3-{cfg.select}_Fold_{fold}.pth", map_location=torch.device('cpu')))
    #model.load_state_dict(torch.load(f"/kaggle/input/commonlit-deberta-hidden-layers-mean/deberta-v3-base_Fold_{fold}.pth", map_location=torch.device('cpu')))
    preds = test_run(model, test_loader)
    final_preds.append(preds)
    del model ; gc.collect()
    torch.cuda.empty_cache()
    # /kaggle/input/commonlit-deberta-base/deberta-v3-large_Fold_0.pth
final_preds_ = np.mean(final_preds, axis=0)
    

******** fold 0 ********


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|          | 0/1 [00:00<?, ?it/s]

******** fold 1 ********


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

...

Some weights of the model checkpoint at /kaggle/input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|          | 0/1 [00:00<?, ?it/s]

******** fold 3 ********


Some weights of the model checkpoint at /kaggle/input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
submission = pd.read_csv('/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv')
submission.head()

,student_id,content,wording
0,000000ffffff,0.0,0.0
1,111111eeeeee,0.0,0.0
2,222222cccccc,0.0,0.0
3,333333dddddd,0.0,0.0


In [21]:
target_cols=['content','wording']
test[target_cols] = final_preds_
submission = submission.drop(columns=target_cols).merge(test[['student_id'] + target_cols], on='student_id', how='left')
display(submission.head())
submission[['student_id'] + target_cols].to_csv('submission.csv', index=False)

,student_id,content,wording
0,000000ffffff,-1.409746,-1.039957
1,111111eeeeee,-1.418963,-1.021244
2,222222cccccc,-1.445152,-1.026454
3,333333dddddd,-1.416630,-1.014911
